In [1]:
def readdata():
    string1=[0,0]
    string2=[0,0]
    string3=[0,0]
    string4=[0,0]
    string5=[0,0]
    finalstring=[0,0,0,0]
    with open('../eppy/eplusout.eso',encoding="utf8") as f:
        csv_reader = csv.reader(f)
        for line_no, line in enumerate(csv_reader, 1):
            #print(line)
            if line[0] == '482':
                string1[0] = line[1]
            elif line[0] == '483':
                string2[0] = line[1]
            elif line[0] == '484':
                string3[0] = line[1]
            elif line[0] == '485':
                string4[0] = line[1]
            elif line[0] == '384':
                string5[0] = line[1]
      
    finalstring=[string1[0] , string2[0],string3[0],string4[0],string5[0]] 
    return finalstring

In [2]:
import numpy as np


#---------------------------- Setting facade parameters-----------------------------

### U-value ###

#HTC=0.01
#HTC=0.1 
#在hihilightk及以后中后续使用的是0.1

#HTCs1=np.arange(1, 101,49)/100   #33
#HTCs2=np.arange(2,11,4)          #4
#HTCs=np.append(HTCs1,HTCs2)
HTCs=[0.005,0.03,0.99,10]
#HTCs=[0.03]
#HTCs=np.arange(3,10,1)/1000

T=0.05 # fixed 5 cm


#Ts=np.arange(1,31,1)/100

### Outer and inner surface infrared emissivity ###

#emi_os=[0.9]
#emi_os=np.arange(1,100,2)/100
emi_os=[0.7,0.9,0.95]

#emi_i=0.8
emi_is=[0.8]


### Outer surface solar reflectivity ###

#refl_os=[0.8]
#refl_os=np.arange(1,100,2)/100
refl_os=[0.3,0.5,0.8,0.95]

### Orientation ###

#Orients=[0]
Orients=[0,45,90]


### Natural ventilation ###
#Natural_vents=[0.6]
Natural_vents=[0,0.6,1.4,2]


### Forced ventilation ###
Forced_vent=0.02  # In Kozai
#Forced_vent=9 #一般用此项
#Forced_vents=np.arange(0,101,5)/10


#------------------------- Setting plant density ----------------------------------
layer=2
#layers=np.arange(1,7,1)
#Frac_plant=0.4
Fracs=[0.2,0.3,0.4]
#Fracs=[0.5]

#------------------------- Setting location and corresponding weather files--------



#-------------------------Setting light parameters---------------------------------

### Unabsorbed fraction of light (related to light reflection loss, can be reduced by adding reflectors) ###
Unabsorbed_radiation_fraction=0.1
#Unabsorbed_radiation_fractions=np.arange(0,31,5)/100

### Setting standard (fixed) light power with different efficiencies and absorbed fractions ###

PPFD=200
#Efficacy=2
Efficacys=[1.8,2,3,3.7]
#Efficacys=[2]


h=6.62*10**(-34)
c=3*10**8
Na=6.02*10**(23)
Fraction_red=0.8
Fraction_blue=0.2
Wav_red=650*10**(-9)
Wav_blue=450*10**(-9)
Ele_light_efficiency=[0,0,0,0]
for ii in range(len(Efficacys)):    
    Ele_light_efficiency[ii]=h*Na*c*(Fraction_red*Efficacys[ii]*10**(-6)/Wav_red+Fraction_blue*Efficacys[ii]*10**(-6)/Wav_blue)
print(Ele_light_efficiency)


### Setting light period ###
Lightperiod=np.array([1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0],dtype='float')

#print(Day_schedule[0])


#--------------------------HVAC properties setting---------------------------------
COP_heating=4
COP_cooling=4
COP_dehumid=4 
COP_nom=5.27
COP_noms=[3]  #3 3.5 4 5.27

#--------------------------Constants in evapotranspiration model of plants---------
Gamma=67
density_mul_capacity_air=1195.35 #20 degree of Celcius air density multiplies capacity

LAI=2.1  #1.5/2.1/0.7 from Song's result
r_s=60*(1500+PPFD)/(200+PPFD)
r_a=400  #100/400
lat_vap=2450

Rnet_fix=129.7  # fixed light input

Saturated_pressure_0_to_30=np.array([0.611,0.657,0.705,0.758,0.813,0.872,0.935,1.001,1.072,1.147,1.227,1.312,1.402,1.497,1.598,1.704,1.817,1.937,2.063,2.196,2.337,2.486,2.643,2.809,2.983,3.167,3.361,3.565,3.78,4.006,4.243,4.493,4.755,5.031,5.32,5.624])
Saturated_pressure_0_to_30=Saturated_pressure_0_to_30*1000
Diff_saturated_pressure_0_to_30=[45,48,51,54,57,61,65,69,73,78,83,88,93,98,104,110,117,123,130,137,145,153,162,170,179,189,199,210,221,232,244,257,269,283,297,312]

[0.36051094153846147, 0.4005677128205127, 0.6008515692307691, 0.7410502687179487]


In [3]:
#------------------Setting plant LP and DP schedule according to Lightperiod----------------------------
#-----------Setting: LP&DP sensible&latent heat, and light period--------------------------------------
from eppy import modeleditor
from eppy.modeleditor import IDF        
iddfile = "../eppy/resources/iddfiles/V22-1-0-Energy+.idd"
fname1 = "../eppy/resources/idffiles/V22-1/20-inch-container-modified-dif-plant-varities.idf"
IDF.setiddname(iddfile)
idf1 = IDF(fname1)
Day_schedule=idf1.idfobjects["Schedule:Day:Hourly"]

Schedule_equipment=[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0]
Schedule_ventilation=[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0]
Schedule_plantlatent_LP=[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0]
Schedule_plantsensible_LP=[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0]
Schedule_plantlatent_DP=[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0]
Schedule_plantsensible_DP=[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0]


In [4]:
#---------------------Setting and plugging in values--------------------------

#iddfile = "../eppy/resources/iddfiles/V22-1-0-Energy+.idd"
#fname1 = "../eppy/resources/idffiles/V22-1/20-inch-container-modified.idf"
#IDF.setiddname(iddfile)
#idf1 = IDF(fname1)
finalstring=[0,0]
material_insu=idf1.idfobjects["MATERIAL"][0]
material_out=idf1.idfobjects["MATERIAL"][1]
material_in=idf1.idfobjects["MATERIAL"][4]
building=idf1.idfobjects['BUILDING'][0]
lightpower=idf1.idfobjects["LIGHTS"][0]
Const_schedule=idf1.idfobjects["Schedule:Constant"]
Naturalvent=idf1.idfobjects["ZoneInfiltration:DesignFlowRate"][0]
OtherEquipment=idf1.idfobjects["OtherEquipment"]
Ele_Equipment=idf1.idfobjects["ElectricEquipment"]
Forcedvent=idf1.idfobjects["ZoneVentilation:DesignFlowRate"][0]
Runperiod=idf1.idfobjects["RunPeriod"][0]

material_insu.Thickness=T
Runperiod.Begin_Month=1
Runperiod.End_Month=12
Runperiod.Begin_Day_of_Month=1
Runperiod.End_Day_of_Month=31
idf1.save()


In [5]:
import csv
import time
import math

# 4处修改，文件名，循环项，循环内容加项，resultdata项修改

months=[1,2,3,4,5,6,7,8,9,10,11,12]
#Locations=["Harbin","Taiyuan","Shanghai","Guangzhou"];
Locations=["Harbin"];
TempRHmodes= ["Tight","Loose"]
#TempRHmodes= ["Loose"]
#LPs=["DAY","NIGHT"]
LPs=["NIGHT"]

Ele_heating_final=0
Ele_cooling_final=0
Ele_dehumid_final=0
Ele_total_final=0
flag=0
Ele_equipment=37.5*3600*24/3600000

filename="../eppy/result-pepper.csv"
results_header=['No.','U(k)','Rsol','Ninfilt','Emi_out',
                'Emi_in','Orient','COPnom','Efficacy','Location','Plantdensity','TempRH','Ele_light','Ele_equip',
                'Ele_heat','Ele_cool','Ele_dehum','Total Electricity']
start_time = time.time()
with open(filename, 'w',encoding="utf-8",newline='') as file:
    writer = csv.writer(file)
    for pp in range(len(HTCs)):
        for qq in range(len(refl_os)):
            for rr in range(len(Natural_vents)):
                for ss in range(len(emi_os)):
                    for tt in range(len(emi_is)):
                        for oo in range(len(Orients)):
                            material_insu.Conductivity=HTCs[pp]
                            material_out.Solar_Absorptance=1-refl_os[qq]
                            Naturalvent.Air_Changes_per_Hour=Natural_vents[rr]
                            material_out.Thermal_Absorptance=emi_os[ss]
                            material_in.Thermal_Absorptance=emi_is[tt]
                            building.North_Axis=Orients[oo]
                            idf1.save()
                            Lightperiod[0:24]=1
                            Lightperiod[10:16]=0 # optimum light period distribution
                            # need to use this version
                            Schedule_lights=Lightperiod    
                            for ii in range(len(Schedule_lights)):
                                if Schedule_lights[ii]==0:
                                    Schedule_plantlatent_LP[ii]= 0
                                    Schedule_plantsensible_LP[ii] = 0
                                    Schedule_plantlatent_DP[ii] = 1
                                    Schedule_plantsensible_DP[ii] = 1
                                else:
                                    Schedule_plantlatent_LP[ii]= 1 * Schedule_lights[ii]
                                    Schedule_plantsensible_LP[ii] = 1 * Schedule_lights[ii]
                                    Schedule_plantlatent_DP[ii] = 1 -  Schedule_lights[ii]
                                    Schedule_plantsensible_DP[ii] = 1 -  Schedule_lights[ii]

                            Day_schedule[2].Hour_1=Schedule_plantlatent_DP[0]
                            Day_schedule[2].Hour_2=Schedule_plantlatent_DP[1]
                            Day_schedule[2].Hour_3=Schedule_plantlatent_DP[2]
                            Day_schedule[2].Hour_4=Schedule_plantlatent_DP[3]
                            Day_schedule[2].Hour_5=Schedule_plantlatent_DP[4]
                            Day_schedule[2].Hour_6=Schedule_plantlatent_DP[5]
                            Day_schedule[2].Hour_7=Schedule_plantlatent_DP[6]
                            Day_schedule[2].Hour_8=Schedule_plantlatent_DP[7]
                            Day_schedule[2].Hour_9=Schedule_plantlatent_DP[8]
                            Day_schedule[2].Hour_10=Schedule_plantlatent_DP[9]
                            Day_schedule[2].Hour_11=Schedule_plantlatent_DP[10]
                            Day_schedule[2].Hour_12=Schedule_plantlatent_DP[11]
                            Day_schedule[2].Hour_13=Schedule_plantlatent_DP[12]
                            Day_schedule[2].Hour_14=Schedule_plantlatent_DP[13]
                            Day_schedule[2].Hour_15=Schedule_plantlatent_DP[14]
                            Day_schedule[2].Hour_16=Schedule_plantlatent_DP[15]
                            Day_schedule[2].Hour_17=Schedule_plantlatent_DP[16]
                            Day_schedule[2].Hour_18=Schedule_plantlatent_DP[17]
                            Day_schedule[2].Hour_19=Schedule_plantlatent_DP[18]
                            Day_schedule[2].Hour_20=Schedule_plantlatent_DP[19]
                            Day_schedule[2].Hour_21=Schedule_plantlatent_DP[20]
                            Day_schedule[2].Hour_22=Schedule_plantlatent_DP[21]
                            Day_schedule[2].Hour_23=Schedule_plantlatent_DP[22]
                            Day_schedule[2].Hour_24=Schedule_plantlatent_DP[23]

                            Day_schedule[3].Hour_1=Schedule_plantsensible_DP[0]
                            Day_schedule[3].Hour_2=Schedule_plantsensible_DP[1]
                            Day_schedule[3].Hour_3=Schedule_plantsensible_DP[2]
                            Day_schedule[3].Hour_4=Schedule_plantsensible_DP[3]
                            Day_schedule[3].Hour_5=Schedule_plantsensible_DP[4]
                            Day_schedule[3].Hour_6=Schedule_plantsensible_DP[5]
                            Day_schedule[3].Hour_7=Schedule_plantsensible_DP[6]
                            Day_schedule[3].Hour_8=Schedule_plantsensible_DP[7]
                            Day_schedule[3].Hour_9=Schedule_plantsensible_DP[8]
                            Day_schedule[3].Hour_10=Schedule_plantsensible_DP[9]
                            Day_schedule[3].Hour_11=Schedule_plantsensible_DP[10]
                            Day_schedule[3].Hour_12=Schedule_plantsensible_DP[11]
                            Day_schedule[3].Hour_13=Schedule_plantsensible_DP[12]
                            Day_schedule[3].Hour_14=Schedule_plantsensible_DP[13]
                            Day_schedule[3].Hour_15=Schedule_plantsensible_DP[14]
                            Day_schedule[3].Hour_16=Schedule_plantsensible_DP[15]
                            Day_schedule[3].Hour_17=Schedule_plantsensible_DP[16]
                            Day_schedule[3].Hour_18=Schedule_plantsensible_DP[17]
                            Day_schedule[3].Hour_19=Schedule_plantsensible_DP[18]
                            Day_schedule[3].Hour_20=Schedule_plantsensible_DP[19]
                            Day_schedule[3].Hour_21=Schedule_plantsensible_DP[20]
                            Day_schedule[3].Hour_22=Schedule_plantsensible_DP[21]
                            Day_schedule[3].Hour_23=Schedule_plantsensible_DP[22]
                            Day_schedule[3].Hour_24=Schedule_plantsensible_DP[23]

                            Day_schedule[6].Hour_1=Schedule_lights[0]
                            Day_schedule[6].Hour_2=Schedule_lights[1]
                            Day_schedule[6].Hour_3=Schedule_lights[2]
                            Day_schedule[6].Hour_4=Schedule_lights[3]
                            Day_schedule[6].Hour_5=Schedule_lights[4]
                            Day_schedule[6].Hour_6=Schedule_lights[5]
                            Day_schedule[6].Hour_7=Schedule_lights[6]
                            Day_schedule[6].Hour_8=Schedule_lights[7]
                            Day_schedule[6].Hour_9=Schedule_lights[8]
                            Day_schedule[6].Hour_10=Schedule_lights[9]
                            Day_schedule[6].Hour_11=Schedule_lights[10]
                            Day_schedule[6].Hour_12=Schedule_lights[11]
                            Day_schedule[6].Hour_13=Schedule_lights[12]
                            Day_schedule[6].Hour_14=Schedule_lights[13]
                            Day_schedule[6].Hour_15=Schedule_lights[14]
                            Day_schedule[6].Hour_16=Schedule_lights[15]
                            Day_schedule[6].Hour_17=Schedule_lights[16]
                            Day_schedule[6].Hour_18=Schedule_lights[17]
                            Day_schedule[6].Hour_19=Schedule_lights[18]
                            Day_schedule[6].Hour_20=Schedule_lights[19]
                            Day_schedule[6].Hour_21=Schedule_lights[20]
                            Day_schedule[6].Hour_22=Schedule_lights[21]
                            Day_schedule[6].Hour_23=Schedule_lights[22]
                            Day_schedule[6].Hour_24=Schedule_lights[23]

                            Day_schedule[9].Hour_1=Schedule_plantlatent_LP[0]
                            Day_schedule[9].Hour_2=Schedule_plantlatent_LP[1]
                            Day_schedule[9].Hour_3=Schedule_plantlatent_LP[2]
                            Day_schedule[9].Hour_4=Schedule_plantlatent_LP[3]
                            Day_schedule[9].Hour_5=Schedule_plantlatent_LP[4]
                            Day_schedule[9].Hour_6=Schedule_plantlatent_LP[5]
                            Day_schedule[9].Hour_7=Schedule_plantlatent_LP[6]
                            Day_schedule[9].Hour_8=Schedule_plantlatent_LP[7]
                            Day_schedule[9].Hour_9=Schedule_plantlatent_LP[8]
                            Day_schedule[9].Hour_10=Schedule_plantlatent_LP[9]
                            Day_schedule[9].Hour_11=Schedule_plantlatent_LP[10]
                            Day_schedule[9].Hour_12=Schedule_plantlatent_LP[11]
                            Day_schedule[9].Hour_13=Schedule_plantlatent_LP[12]
                            Day_schedule[9].Hour_14=Schedule_plantlatent_LP[13]
                            Day_schedule[9].Hour_15=Schedule_plantlatent_LP[14]
                            Day_schedule[9].Hour_16=Schedule_plantlatent_LP[15]
                            Day_schedule[9].Hour_17=Schedule_plantlatent_LP[16]
                            Day_schedule[9].Hour_18=Schedule_plantlatent_LP[17]
                            Day_schedule[9].Hour_19=Schedule_plantlatent_LP[18]
                            Day_schedule[9].Hour_20=Schedule_plantlatent_LP[19]
                            Day_schedule[9].Hour_21=Schedule_plantlatent_LP[20]
                            Day_schedule[9].Hour_22=Schedule_plantlatent_LP[21]
                            Day_schedule[9].Hour_23=Schedule_plantlatent_LP[22]
                            Day_schedule[9].Hour_24=Schedule_plantlatent_LP[23]

                            Day_schedule[10].Hour_1=Schedule_plantsensible_LP[0]
                            Day_schedule[10].Hour_2=Schedule_plantsensible_LP[1]
                            Day_schedule[10].Hour_3=Schedule_plantsensible_LP[2]
                            Day_schedule[10].Hour_4=Schedule_plantsensible_LP[3]
                            Day_schedule[10].Hour_5=Schedule_plantsensible_LP[4]
                            Day_schedule[10].Hour_6=Schedule_plantsensible_LP[5]
                            Day_schedule[10].Hour_7=Schedule_plantsensible_LP[6]
                            Day_schedule[10].Hour_8=Schedule_plantsensible_LP[7]
                            Day_schedule[10].Hour_9=Schedule_plantsensible_LP[8]
                            Day_schedule[10].Hour_10=Schedule_plantsensible_LP[9]
                            Day_schedule[10].Hour_11=Schedule_plantsensible_LP[10]
                            Day_schedule[10].Hour_12=Schedule_plantsensible_LP[11]
                            Day_schedule[10].Hour_13=Schedule_plantsensible_LP[12]
                            Day_schedule[10].Hour_14=Schedule_plantsensible_LP[13]
                            Day_schedule[10].Hour_15=Schedule_plantsensible_LP[14]
                            Day_schedule[10].Hour_16=Schedule_plantsensible_LP[15]
                            Day_schedule[10].Hour_17=Schedule_plantsensible_LP[16]
                            Day_schedule[10].Hour_18=Schedule_plantsensible_LP[17]
                            Day_schedule[10].Hour_19=Schedule_plantsensible_LP[18]
                            Day_schedule[10].Hour_20=Schedule_plantsensible_LP[19]
                            Day_schedule[10].Hour_21=Schedule_plantsensible_LP[20]
                            Day_schedule[10].Hour_22=Schedule_plantsensible_LP[21]
                            Day_schedule[10].Hour_23=Schedule_plantsensible_LP[22]
                            Day_schedule[10].Hour_24=Schedule_plantsensible_LP[23]
                            idf1.save()
                            for cc in range(len(Locations)):
                                if cc+1 ==1:
                                    epwfile = "../eppy/resources/idffiles/V22-1/CHN_Heilongjiang.Harbin.509530_CSWD.epw"
                                elif cc+1 == 2:
                                    epwfile = "../eppy/resources/idffiles/V22-1/CHN_Shanxi.Taiyuan.537720_CSWD.epw"
                                elif cc+1 == 3:
                                    epwfile = "../eppy/resources/idffiles/V22-1/CHN_Shanghai.Shanghai.583620_CSWD.epw"
                                elif cc + 1 == 4:
                                    epwfile = "../eppy/resources/idffiles/V22-1/CHN_Guangdong.Guangzhou.592870_CSWD.epw"

                                for ff in range(len(TempRHmodes)):
                                    if ff + 1 == 3:
                                        Tmin=20;
                                        Tmax=22;
                                        RHmin=0.6
                                        RHmax=0.7
                                    else:
                                        Tmin= 16;  #25/18 
                                        Tmax= 22
                                        RHmin=0.5
                                        RHmax=0.95
                                    Const_schedule[5].Hourly_Value=Tmin
                                    Const_schedule[6].Hourly_Value=Tmax
                                    Const_schedule[8].Hourly_Value=RHmin*100
                                    Const_schedule[9].Hourly_Value=RHmax*100
                                    idf1.save()
                                    for bb in range(len(Efficacys)):                                        
                                        for dd in range(len(Fracs)):
                                            ### Calculating plant evapotranspiration
                                            Rnet=Rnet_fix # electricity of light
                                            Ele_light_efficiency=h*Na*c*(Fraction_red*Efficacys[bb]*10**(-6)/Wav_red+Fraction_blue*Efficacys[bb]*10**(-6)/Wav_blue)
                                            Ele_power_light=Rnet/(1-Unabsorbed_radiation_fraction)/(Ele_light_efficiency)
                                            #print(Rnet)
                                            #print(Ele_light_efficiency)
                                            #print(Ele_power_light)
                                            
                                            Lightpower_input=layer*Fracs[dd]*(Ele_power_light*(1-Ele_light_efficiency)+Ele_power_light*\
                                                    Ele_light_efficiency*Unabsorbed_radiation_fraction)
                                            Visible_Fraction_input=layer*Fracs[dd]*(Ele_power_light*\
                                                    Ele_light_efficiency*Unabsorbed_radiation_fraction)/Lightpower_input
                                            #print(Lightpower_input)
                                            #print(Visible_Fraction_input)
                                            lightpower.Watts_per_Zone_Floor_Area=Lightpower_input
                                            lightpower.Fraction_Visible=Visible_Fraction_input

                                            Q_latent=151.13 #fixed ET from literature
                                            # 假设光期的蒸发，湿度是最小湿度和最大湿度的四等分点的第三个点
                                            Q_sensible=Rnet-Q_latent
                                            Q_latent_DP=0.02*Q_latent # based on the validation experiment by Yu Zhitong
                                            Q_sensible_DP=-Q_latent_DP
                                            
                                            Q_latent_LP_input=Q_latent*layer*Fracs[dd]
                                            Q_sensible_LP_input=Q_sensible*layer*Fracs[dd]
                                            Q_latent_DP_input=Q_latent_DP*layer*Fracs[dd]
                                            Q_sensible_DP_input=Q_sensible_DP*layer*Fracs[dd]
                                            
                                            #print(Q_latent_LP_input)
                                            #print(Q_sensible_LP_input)
                                            #print(Q_latent_DP_input)
                                            #print(Q_sensible_DP_input)
                                            
                                             #OtherEquipment[0].Design_Level=2.4
                                            OtherEquipment[1].Power_per_Zone_Floor_Area=Q_latent_DP_input
                                            OtherEquipment[2].Power_per_Zone_Floor_Area=Q_sensible_DP_input
                                            OtherEquipment[3].Power_per_Zone_Floor_Area=Q_latent_LP_input
                                            OtherEquipment[4].Power_per_Zone_Floor_Area=Q_sensible_LP_input

                                            Ele_light=Ele_power_light*layer*Fracs[dd]*5.88*2.33*18/1000
                                            
                                            #print(Ele_light)
                                            idf1.save()

                                            for aa in range(len(COP_noms)):

                                                iddfile = "../eppy/resources/iddfiles/V22-1-0-Energy+.idd"
                                                IDF.setiddname(iddfile)
                                                idfname = "../eppy/resources/idffiles/V22-1/20-inch-container-modified-dif-plant-varities.idf"

                                                idf = IDF(idfname, epwfile)
                                                idf.run()
                                                #print(Day_schedule[0])
                                                finalstring=readdata()

                                                #print(finalstring)
                                                #Energy_heating=(eval(finalstring[1]))/365/3600000
                                                #Energy_cooling=(eval(finalstring[3])-eval(finalstring[2]))/365/3600000
                                                #Energy_dehumid=(eval(finalstring[0])+eval(finalstring[2]))/365/3600000
                                                #Energy_dehumid=(eval(finalstring[2]))/365/3600000
                                                #Energy_total=Energy_heating+Energy_cooling+Energy_dehumid
                                                #Ambient_temp=eval(finalstring[4])
                                                Ele_heating= (eval(finalstring[1])-eval(finalstring[0]))/3600000/COP_noms[aa]
                                                Ele_cooling= (eval(finalstring[3])-eval(finalstring[2]))/3600000/\
                                                 COP_noms[aa]
                                                Ele_dehumid= (eval(finalstring[0])+eval(finalstring[2]))/3600000/\
                                                 COP_noms[aa]
                                                Ele_cooling_final=Ele_cooling
                                                Ele_heating_final=Ele_heating
                                                Ele_dehumid_final=Ele_dehumid

                                                Ele_total_final=Ele_heating_final/365+Ele_cooling_final/365+\
                                                    Ele_dehumid_final/365+Ele_light+Ele_equipment
                                                results_data=[flag+1,HTCs[pp],refl_os[qq],Natural_vents[rr],emi_os[ss],emi_is[tt],
                                                    Orients[oo],COP_noms[aa],Efficacys[bb],Locations[cc],layer*Fracs[dd],TempRHmodes[ff],
                                                              Ele_light,Ele_equipment,Ele_heating_final/365,
                                                    Ele_cooling_final/365, Ele_dehumid_final/365,Ele_total_final]

                                                Ele_heating_final=0
                                                Ele_cooling_final=0
                                                Ele_dehumid_final=0
                                                Ele_total_final=0
                                                # 3. Write data to the file
                                                if flag==0:
                                                    writer.writerow(results_header)
                                                writer.writerow(results_data)
                                                flag=flag+1


end_time = time.time()   # 记录结束时间
run_time = end_time - start_time
#print(run_time)


C:/EnergyPlusV22-1-0\energyplus.exe --weather D:\PhD career\2023.2-2024.1 career\Simulation\eppy\resources\idffiles\V22-1\CHN_Heilongjiang.Harbin.509530_CSWD.epw --output-directory D:\PhD career\2023.2-2024.1 career\Simulation\eppy --idd D:\PhD career\2023.2-2024.1 career\Simulation\eppy\resources\iddfiles\V22-1-0-Energy+.idd D:\PhD career\2023.2-2024.1 career\Simulation\eppy\resources\idffiles\V22-1\20-inch-container-modified-dif-plant-varities_ce916c.idf


C:/EnergyPlusV22-1-0\energyplus.exe --weather D:\PhD career\2023.2-2024.1 career\Simulation\eppy\resources\idffiles\V22-1\CHN_Heilongjiang.Harbin.509530_CSWD.epw --output-directory D:\PhD career\2023.2-2024.1 career\Simulation\eppy --idd D:\PhD career\2023.2-2024.1 career\Simulation\eppy\resources\iddfiles\V22-1-0-Energy+.idd D:\PhD career\2023.2-2024.1 career\Simulation\eppy\resources\idffiles\V22-1\20-inch-container-modified-dif-plant-varities_0677fa.idf


C:/EnergyPlusV22-1-0\energyplus.exe --weather D:\PhD career\2023.2-2024.1

KeyboardInterrupt: 